<a href="https://colab.research.google.com/github/shweta-mahalingam/Potato-disease-classification/blob/main/Potato_plant_classification_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving archive(1).zip to archive(1).zip


In [ ]:
import zipfile
import os

In [ ]:
file_name = "archive(1).zip"
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall("/content")

os.listdir("/content")
print("Files extracted:")
print(os.listdir("/content"))

Files extracted:
['.config', 'Potato Plant Disease', 'archive(1).zip', 'drive', 'sample_data']


In [ ]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50

In [ ]:
  dataset = tf.keras.preprocessing.image_dataset_from_directory(
      "Potato Plant Disease",
      shuffle = True,
      image_size = (IMAGE_SIZE,IMAGE_SIZE),
      batch_size = BATCH_SIZE
  )

Found 2152 files belonging to 3 classes.


In [ ]:
class_name = dataset.class_names
class_name

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [ ]:
for image_batch,label_batch in dataset.take(1):
  print(image_batch.shape)
  print(len(label_batch.numpy()))

(32, 256, 256, 3)
32


In [ ]:
plt.figure(figsize=(10,10))
for image_batch,label_batch in dataset.take(1):
  for i in range(12):
    ax = plt.subplot(3,4,i+1)
    plt.imshow(image_batch[i].numpy().astype("uint8"))
    plt.axis("off")
    plt.title(class_name[label_batch[i]])

In [ ]:
len(dataset)

68

In [ ]:
#80% ==> training
#20% ==> 10% validation, 10% test

In [ ]:
def ds_split(ds,train_split=0.8,val_split=0.1,test_split=0.1,shuffle=True,shuffle_size=10000):
  if shuffle:
    ds = ds.shuffle(shuffle_size,seed=12)

  train_size = int(train_split * len(ds))
  val_size = int(val_split * len(ds))


  train_ds = ds.take(train_size)
  val_ds = ds.skip(train_size).take(val_size)
  test_ds = ds.skip(train_size).skip(val_size)

  return train_ds,val_ds,test_ds

In [ ]:
train_ds,val_ds,test_ds = ds_split(dataset)

In [ ]:
len(train_ds),len(val_ds),len(test_ds)

(54, 6, 8)

In [ ]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE,),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

**MODEL**

In [ ]:
input_shape = (BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32,(3,3),activation="relu",input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size = (3,3),activation="relu"),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation="relu"),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation="relu"),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation="relu"),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation="relu"),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(n_classes, activation = 'softmax')

])

model.build(input_shape = input_shape)

In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 sequential_6 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 conv2d_24 (Conv2D)          (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d_24 (MaxPooli  (32, 127, 127, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_25 (Conv2D)          (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_25 (MaxPooli  (32, 62, 62, 64)          0         
 ng2D)                                                

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics = ['accuracy']

)

In [ ]:
history = model.fit(
    train_ds,
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    verbose = 1,
    validation_data = val_ds
)

Epoch 1/50
54/54 [==============================] - 228s 4s/step - loss: 0.9281 - accuracy: 0.4519 - val_loss: 0.8718 - val_accuracy: 0.4531
Epoch 2/50
54/54 [==============================] - 211s 4s/step - loss: 0.7185 - accuracy: 0.6755 - val_loss: 0.5042 - val_accuracy: 0.7798
Epoch 3/50
54/54 [==============================] - 223s 4s/step - loss: 0.4327 - accuracy: 0.8212 - val_loss: 0.3910 - val_accuracy: 0.8385
Epoch 4/50
54/54 [==============================] - 210s 4s/step - loss: 0.3251 - accuracy: 0.8609 - val_loss: 0.6403 - val_accuracy: 0.7865
Epoch 5/50
54/54 [==============================] - 214s 4s/step - loss: 0.3042 - accuracy: 0.8791 - val_loss: 0.3347 - val_accuracy: 0.8646
Epoch 6/50
54/54 [==============================] - 219s 4s/step - loss: 0.2560 - accuracy: 0.8955 - val_loss: 0.2463 - val_accuracy: 0.9115
Epoch 7/50
54/54 [==============================] - 209s 4s/step - loss: 0.2423 - accuracy: 0.9014 - val_loss: 0.1792 - val_accuracy: 0.9271
Epoch 8/50
54

In [ ]:
model.save("model.keras")

In [ ]:
scores = model.evaluate(test_ds)
scores

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

In [ ]:
EPOCHS = 5
plt.figure(figsize = (8,8))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS), acc, label = 'Training accuracy')
plt.plot(range(EPOCHS), val_acc, label = 'Validation accuracy')
plt.legend(loc = 'lower right')
plt.title('Training and Validation Accuracy')

In [ ]:
import numpy as np
for image_batch,label_batch in test_ds.take(1):
    first_image = images_batch[0].numpy().astype("uint8")
    first_label = labels_batch[0].numpy()
    print("first image to print")
    plt.imshow(first_image)
    print("actual label is:",class_name[first_label])
    batch_prediction = model.predict(images_batch)
    print("predicted_label is:",class_name[np.argmax(batch_prediction[0])])